In [ ]:
"""
Created on Mon Feb 10 21:30:54 2020

@author: IULEN
"""

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import pathlib

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from keras.models import Model

In [ ]:
print(tf.__version__)

In [ ]:
column_names = ['Width of the haptic','Haptic length','Start of the haptic curvature','Junction',
                'Angle of the haptic','Thickness', 'close',
                'Reaction Force', 'Axial Displacement', 'Rotation','Tilt', 'Strain Energy']

In [ ]:
dataset = pd.read_csv("magia2.csv", names=column_names, index_col=False,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True)

In [ ]:
dataset.tail()

In [ ]:
dataset.isna().sum()
dataset = dataset.dropna()

In [ ]:
#Más preprocesado
dataset = dataset.drop(dataset[dataset['Reaction Force']<1].index)
dataset.pop('Haptic length')
dataset.pop('Angle of the haptic')
#dataset.pop('Rotation')
#dataset.pop('Tilt')
#dataset.pop('Reaction Force')
#dataset.pop('Strain Energy')

In [ ]:
#train_dataset = dataset
dataset.describe().transpose()

In [ ]:
data_label = dataset.copy()

In [ ]:
train_labels1 = data_label.pop('Reaction Force')
train_labels4 = data_label.pop('Tilt')
train_labels3 = data_label.pop('Rotation')
train_labels5 = data_label.pop('Strain Energy')
train_labels2 = data_label.pop('Axial Displacement')

In [ ]:
output_label = pd.concat([train_labels1, train_labels2, train_labels3 , train_labels4 , train_labels5], axis=1,)

train_label.pop('Rotation')
test_label.pop('Rotation')


In [ ]:
data_stats = data_label.describe().transpose()

In [ ]:
def normalizingInput(x):
  return (x - data_stats['min']) / (data_stats['max'] - data_stats['min'])

def desnormalizingInput(x):
 return (x *  (data_stats['max'] - data_stats['min']) + data_stats['min'])
output_stats = output_label.describe().transpose()
def normalizingOutput(x):
 return (x - output_stats['min']) / (output_stats['max'] - output_stats['min'])
def desnormalizingOutput(x):
 return (x *  (output_stats['max'] - output_stats['min']) + output_stats['min'])

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

sns.pairplot(train_dataset[['Reaction Force', 'Axial Displacement', 'Rotation','Tilt', 
                                'Strain Energy']], diag_kind="kde")
sns.pairplot(train_dataset[['Width of the haptic','Start of the haptic curvature','Junction',
                               'Thickness']], diag_kind="kde")

In [ ]:
train_labels1 = train_dataset.pop('Reaction Force')
test_labels1 = test_dataset.pop('Reaction Force')
train_labels4 = train_dataset.pop('Tilt')
test_labels4 = test_dataset.pop('Tilt')
train_labels3 = train_dataset.pop('Rotation')
test_labels3 = test_dataset.pop('Rotation')
train_labels5 = train_dataset.pop('Strain Energy')
test_labels5 = test_dataset.pop('Strain Energy')
train_labels2 = train_dataset.pop('Axial Displacement')
test_labels2 = test_dataset.pop('Axial Displacement')

In [ ]:
train_label = pd.concat([train_labels1, train_labels2, train_labels3 , train_labels4 , train_labels5], axis=1,)
test_label = pd.concat([test_labels1, test_labels2 ,test_labels3, test_labels4 , test_labels5], axis=1,)

In [ ]:
norm_data_train = normalizingInput(train_dataset)
#norm_output_train = normalizingOutput(train_label)
norm_data_test = normalizingInput(test_dataset)
#norm_output_test = normalizingOutput(test_label)

In [ ]:
def custom_loss(y_true, y_pred):

    # I am pretty sure this does not wor
    mse_loss = y_true-y_pred
#    cross_entropy_loss = K.dot(K.transpose(label_prob_dist), output_prob_dist)

    return y_true[:1]

In [ ]:
def custom_loss2(y_true, y_pred):

    # I am pretty sure this does not wor
    mse_loss = y_true-y_pred
#    cross_entropy_loss = K.dot(K.transpose(label_prob_dist), output_prob_dist)

    return y_pred[:2]

out_dim = train_label.shape[1]

In [ ]:
model = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[len(norm_data_train.keys())]),
    layers.Dropout(0.3),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(5),
    ])

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model = Model(inputs=norm_data_train.keys(), outputs=train_label.keys())

In [ ]:
model.compile(loss='mae',
                optimizer=optimizer,
                metrics=['mae', custom_loss, custom_loss2],
#                loss_weights={'Reaction Force':0.1, 'Axial Displacement':1, 'Rotation':1,'Tilt':1, 'Strain Energy':1}
                )

model = keras.layers.input(inputs=norm_data_train.keys())
model = keras.layers.outputs(train_label.keys())

In [ ]:
#model = build_model()
model.summary()

example_batch = norm_data_train[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  plt.figure(figsize=(12,8))
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [All responses]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
#  plt.ylim([0,0.2])
  plt.legend()

 plt.figure()
 plt.xlabel('Epoch')
 plt.ylabel('Mean Abs Error [MPG]')
 plt.plot(hist['epoch'], hist['mape'],
          label='Train Error')
 plt.plot(hist['epoch'], hist['val_mape'],
          label = 'Val Error')
 plt.ylim([0,100000000])
 plt.legend()

In [ ]:
  plt.figure(figsize=(12,8))
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [All responses]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
#  plt.ylim([0,0.1])
  plt.legend()
  plt.show()

In [ ]:
# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
EPOCHS = 50

In [ ]:
history = model.fit(norm_data_train, train_label, epochs=EPOCHS, batch_size=128,
                    validation_split = 0.2, verbose=1, callbacks=[PrintDot()])
#history = model.fit(norm_data_train, norm_output_train, epochs=EPOCHS, batch_size=64,
#                    validation_split = 0.2, verbose=1, callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_label, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

% mean_squared_error(ytest[:,0], ypred[:,0])
print("Testing set Mean Abs Error: {:5.2f} MPG" % mean_squared_error(test_label[:,1]))

In [ ]:
test_predictions = model.predict(norm_data_test)

In [ ]:
Response_names = ['Reaction Force', 'Tilt', 'Rotation','Strain Energy', 'Axial Displacement']
test_predictions=pd.DataFrame (test_predictions, columns = Response_names)

In [ ]:
test_Predictions=desnormalizingOutput(test_predictions)

In [ ]:
plt.figure(figsize=(12,8))
name=['Tilt']
plt.scatter(test_label[name], test_Predictions[name])
plt.xlabel('True Values'+ str(name))
plt.ylabel('Predictions' + str(name))
plt.axis('equal')
plt.axis('square')
#plt.xlim([0,plt.xlim()[1]])
#plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-1000, 1000], [-1000, 1000])

In [ ]:
error = (test_Predictions[name].to_numpy() - test_label[name].to_numpy() )
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

In [ ]:
# Saves the entire model into a file named as 'dnn_model.h5'
model.save('C-loop_v1.h5')
# Later, (maybe another day), you can load the trained model for prediction.
#model = load_model('dnn_model.h5')